# 함수형 API 다중입출력모델

In [69]:
from tensorflow import keras
from keras import Input
from keras.layers import Dense
from keras.layers import Concatenate
from keras.models import Model
import numpy as np

## 1. 데이터 준비

-- 입력 데이터 --  
- 이슈 티켓의 제목 (텍스트)  
- 이슈 티켓의 텍스트 본문 (텍스트)  
- 사용자가 추가한 태그 (범주에서 선택 (multi-hot 인코딩되었다고 가정))

In [70]:
vocab_size = 10000	# 단어 개수
num_tags = 100		# 태그 개수
num_departments = 4	# 부서 개수
num_samples = 1280	# 데이터 개수 (이슈 티켓)

In [71]:
# 함수형 API 모델 설계 테스트이기 때문에 데이터는 더미데이터를 생성

title_data = np.random.randint(0,2, size=(num_samples, vocab_size))
text_body_data = np.random.randint(0,2, size=(num_samples, vocab_size))
tags_data = np.random.randint(0,2, size=(num_samples, num_tags))

priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0,2, size=(num_samples, num_departments))

## 2. 모델 설계

### 2-1. Input Layer 정의

In [72]:
title = Input(shape=(vocab_size,), name="title")
text_body = Input(shape=(vocab_size,), name="text_body")
tags = Input(shape=(num_tags,), name="tags")

### 2-2. Hidden Layer 정의

In [73]:
# input layer들의 출력을 하나로 모으는 Layer
features = Concatenate()([title, text_body, tags])

# Concatenate에 Dense Layer를 붙여서 재정의
features = Dense(64, activation="relu")(features)

### 2-3. Output Layer 정의

In [74]:
# 각각의 Output Layer에 Hidden를 붙여 정의
priority = Dense(1, activation="sigmoid", name="priority")(features)
department = Dense(num_departments, activation="softmax", name="department")(features)

### 2-4. 모델 객체 생성

inputs와 outputs는 Concatenate, output을 정의한 순서대로 설정해야 함.

In [75]:
model = Model(inputs=[title, text_body, tags], outputs=[priority, department])

## 3. 모델 학습

### 3-1. 모델 컴파일

**Loss** Function에는 Output Layer 순서대로 List형태로 설정  
**Metrics**는 하나의 Output Layer에 여러 개의 Metric이 지정될 수 있으므로 각각의 Output Layer에 List형태로 설정

In [76]:
model.compile(loss=["mse", "categorical_crossentropy"], optimizer="rmsprop", metrics=[["mae"],["accuracy"]])

아래와 같이 Dictionary 형태로 순서 상관없이 지정할 수 있음.

In [ ]:
'''
model.compile(loss={"priority": "mse",
					"department": "categorical_crossentropy"},
				optimizer="rmsprop", 
				metrics={"priority": ["mae"],
			 			"department": ["accuracy"]})
'''

### 3-2. 모델 학습

Model 구조에 맞게 Input/Output Data를 순서대로 넣어줘야 함.

In [77]:
history = model.fit([title_data, text_body_data, tags_data],
		  [priority_data, department_data],
		   epochs=1)

40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - department_accuracy: 0.3187 - department_loss: 51.8780 - loss: 52.2062 - priority_loss: 0.3282 - priority_mae: 0.4932


## 4. 모델 평가

model.fit()과 마찬가지로 Data의 순서를 지켜야 함.

In [78]:
e = model.evaluate([title_data, text_body_data, tags_data],
		  [priority_data, department_data])

print(e)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - department_accuracy: 0.2445 - department_loss: 31.7360 - loss: 32.0698 - priority_loss: 0.3337 - priority_mae: 0.4986
[32.06977081298828, 0.33373427391052246, 31.736032485961914, 0.24453124403953552, 0.4986289441585541]


## 5. 모델 추론

마찬가지로 순서 지켜서 Data 설정.  
Output이 2개이므로 반환값도 2개임.

In [ ]:
priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])
print(priority_preds)
print(department_preds)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
[array([[2.4158909e-24],
       [1.7698106e-24],
       [1.2391049e-24],
       ...,
       [2.1350927e-24],
       [1.7620906e-24],
       [4.2166804e-24]], shape=(1280, 1), dtype=float32), array([[8.0217175e-09, 1.0000000e+00, 8.0122549e-12, 1.4425713e-10],
       [2.2759879e-08, 1.0000000e+00, 1.3349386e-11, 3.0557050e-11],
       [2.3387951e-08, 1.0000000e+00, 2.1228352e-11, 1.3043686e-10],
       ...,
       [7.9692313e-08, 9.9999988e-01, 1.9411391e-11, 4.5067991e-10],
       [1.1238113e-08, 1.0000000e+00, 1.1852915e-11, 1.0118829e-10],
       [2.2798810e-08, 1.0000000e+00, 2.2230833e-11, 1.6292687e-10]],
      shape=(1280, 4), dtype=float32)]
